geo-hls-remove-clouds

Removes clouds from HLS data

In [ ]:
!pip install xarray matplotlib geopandas rioxarray numpy shapely rasterio pyproj ipython dask distributed jinja2 bokeh

In [1]:
import os
from glob import glob

import matplotlib.pyplot as plt
import geopandas as gpd
import rasterio as rio
import xarray as xr
import rioxarray as rxr
import numpy as np
import glob
from shapely.geometry import mapping
import pyproj
from pyproj import Proj
from shapely.ops import transform
import logging
import re
import sys
from dask.distributed import Client


In [2]:
# path for input
input_dir=os.environ.get('input_dir','/home/romeokienzler/Downloads/HLS2022/HLS/**/')

# path for output
output_dir=os.environ.get('output_dir','/home/romeokienzler/Downloads/HLSS30.CF2/')

# file filter pattern
file_filter_pattern=os.environ.get('file_filter_pattern','HLS.S30*0.B*tif')
file_filter_pattern='HLS.S30.T15STU.2022171T165901.v2.0.B01.tif'

In [3]:
root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)

parameters = list(
    map(lambda s: re.sub('$', '"', s),
        map(
            lambda s: s.replace('=', '="'),
            filter(
                lambda s: s.find('=') > -1 and bool(re.match(r'[A-Za-z0-9_]*=[.\/A-Za-z0-9]*', s)),
                sys.argv
            )
    )))

for parameter in parameters:
    logging.debug('Parameter: ' + parameter)
    exec(parameter)

In [4]:
client = Client(n_workers=8, threads_per_worker=1, memory_limit='8GB')
client

2023-06-21 17:02:30,236 - asyncio - DEBUG - Using selector: EpollSelector


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 8,Total memory: 59.60 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45941,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 59.60 GiB
Comm: tcp://127.0.0.1:35039,Total threads: 1
Dashboard: http://127.0.0.1:42253/status,Memory: 7.45 GiB
Nanny: tcp://127.0.0.1:43853,


In [5]:
def open_clean_band(band_path, crop_layer=None):
    """A function that opens a Landsat/Sentinel band as an (rio)xarray object

    Parameters
    ----------
    band_path : list
        A list of paths to the tif files that you wish to combine.

    crop_layer : geopandas geodataframe
        A geodataframe containing the clip extent of interest. NOTE: this will 
        fail if the clip extent is in a different CRS than the raster data.

    Returns
    -------
    An single xarray object with the Landsat band data.

    """

    if crop_layer is not None:
        try:
            clip_bound = crop_layer.geometry
            cleaned_band = rxr.open_rasterio(band_path,
                                             masked=True).rio.clip(clip_bound,
                                                                   from_disk=True).squeeze()
        except Exception as err:
            print("Oops, I need a geodataframe object for this to work.")
            print(err)
    else:
        cleaned_band = rxr.open_rasterio(band_path,
                                         masked=True).squeeze()

    return cleaned_band


def process_bands(paths, crop_layer=None, stack=False):
    """
    Open, clean and crop a list of raster files using rioxarray.

    Parameters
    ----------
    paths : list
        A list of paths to raster files that could be stacked (of the same 
        resolution, crs and spatial extent).

    crop_layer : geodataframe
        A geodataframe containing the crop geometry that you wish to crop your
        data to.

    stack : boolean
        If True, return a stacked xarray object. If false will return a list
        of xarray objects.

    Returns
    -------
        Either a list of xarray objects or a stacked xarray object
    """

    all_bands = []
    for i, aband in enumerate(paths):
        cleaned = open_clean_band(aband, crop_layer)
        cleaned["band"] = i+1
        all_bands.append(cleaned)

    if stack:
        print("I'm stacking your data now.")
        return xr.concat(all_bands, dim="band")
    else:
        print("Returning a list of xarray objects.")
        return all_bands

def good_quality(list_of_files, list_of_masks):
    """
    Open list of files and the corresponding list of masks for a bive satellite and remove all pixels that are cloud contaimated as flagged by the cloud mask
    Parameters
    ----------
    list of files : list
        A list of paths to files that could be stacked (of the same 
        resolution, crs and spatial extent).

    list-of_masks : list
        A list od ptahs to the maks for  each of the files listed above

    Returns
    -------
        Either a pixel values that are valid for a given satellite.
    """
    bitword_order = (1, 1, 1, 1, 1, 1, 2)  # set the number of bits per bitword
    num_bitwords = len(bitword_order)      # Define the number of bitwords based on your input above
    total_bits = sum(bitword_order)        # Should be 8, 16, or 32 depending on datatype
    list_of_files1.sort()
    list_of_masks1.sort()
    
    stacked_list=process_bands(list_of_files1,crop_layer=None, stack=True)
    stacked_masks=process_bands(list_of_masks1,crop_layer=None, stack=True)
            
    qVals = list(np.unique(stacked_masks))  # Create a list of unique values that need to be converted to binary and decoded
    qVals = [0 if x != x else x for x in qVals]
            
    all_bits = list()
    goodQuality = []
    for v in qVals:
        all_bits = []
        bits = total_bits
        i = 0

    # Convert to binary based on the values and # of bits defined above:
        bit_val = format(int(v), "b").zfill(bits)
       #print('\n' + str(v) + ' = ' + str(bit_val))
        all_bits.append(str(v) + ' = ' + str(bit_val))

    # Go through & split out the values for each bit word based on input above:
        for b in bitword_order:
            prev_bit = bits
            bits = bits - b
            i = i + 1
            if i == 1:
                bitword = bit_val[bits:]
               #print(' Bit Word ' + str(i) + ': ' + str(bitword))
                all_bits.append(' Bit Word ' + str(i) + ': ' + str(bitword)) 
            elif i == num_bitwords:
                bitword = bit_val[:prev_bit]
                #print(' Bit Word ' + str(i) + ': ' + str(bitword))
                all_bits.append(' Bit Word ' + str(i) + ': ' + str(bitword))
            else:
                bitword = bit_val[bits:prev_bit]
                #print(' Bit Word ' + str(i) + ': ' + str(bitword))
                all_bits.append(' Bit Word ' + str(i) + ': ' + str(bitword))

    # 2, 4, 5, 6 are the bits used. All 4 should = 0 if no clouds, cloud shadows were present, and pixel is not snow/ice/water
        if int(all_bits[2].split(': ')[-1]) + int(all_bits[4].split(': ')[-1]) + \
        int(all_bits[5].split(': ')[-1]) + int(all_bits[6].split(': ')[-1]) + int(all_bits[3].split(': ')[-1])==0: #
            goodQuality.append(v)
    return goodQuality

In [6]:
import glob
list_of_files=[]
list_of_tiles=[]
list_of_bands=[]

#input_dir='/data_external/HLS/CA/S30/2022/11/S/L/B/**/'
for f in glob.glob(os.path.join(input_dir+file_filter_pattern), recursive=True):
    logging.debug(f)
    if f[-34:-28] not in list_of_tiles:
        list_of_tiles.append(f[-34:-28])
    if f[-7:-4] not in list_of_bands:
        list_of_bands.append(f[-7:-4])
    list_of_files.append(f)

logging.debug('list_of_files' + str(list_of_files))
logging.debug('list_of_tiles' + str(list_of_tiles))
logging.debug('list_of_bands' + str(list_of_bands))


2023-06-21 17:11:09,324 - root - DEBUG - /home/romeokienzler/Downloads/HLS2022/HLS/month=06/HLS.S30.T15STU.2022171T165901.v2.0.B01.tif
2023-06-21 17:11:09,329 - root - DEBUG - list_of_files['/home/romeokienzler/Downloads/HLS2022/HLS/month=06/HLS.S30.T15STU.2022171T165901.v2.0.B01.tif']
2023-06-21 17:11:09,332 - root - DEBUG - list_of_tiles['T15STU']
2023-06-21 17:11:09,334 - root - DEBUG - list_of_bands['B01']


In [7]:
list_of_masks=[]
for f in glob.glob(os.path.join(input_dir+'HLS.S30*Fmask.tif'), recursive=True):
    #print(f,f[-9:-4],f[-34:-28])
    list_of_masks.append(f)

In [8]:
list_of_files1=[]
list_of_masks1=[]


for j in list_of_tiles:
        for i in list_of_bands:
            list_of_files1=[]
            list_of_masks1=[]
            for f in glob.glob(os.path.join(input_dir+'HLS.S30.'+str(j)+'*'+str(i)+'.tif'), recursive=True):
                list_of_files1.append(f)
            for f in glob.glob(os.path.join(input_dir+'HLS.S30.'+str(j)+'*Fmask.tif'), recursive=True):    
                list_of_masks1.append(f)
            
            goodQuality= good_quality(list_of_files1, list_of_masks1)
            
            stacked_list=process_bands(list_of_files1,crop_layer=None, stack=True)
            stacked_masks=process_bands(list_of_masks1,crop_layer=None, stack=True)
            cf_band_all = np.ma.MaskedArray(stacked_list, np.in1d(stacked_masks, goodQuality, invert=True))  # Apply QA mask to the EVI data
            cf_band_all = np.ma.filled(cf_band_all, np.nan)
            cf_band_all[np.isnan(cf_band_all)] = 0
            cf=cf_band_all[0]
            #sequentially pick cloud free pixel values 
            for i in np.arange(len(stacked_list)-1):  #remove the cloud contaminated pixels
                cf=xr.where((cf==0),cf_band_all[i+1],cf)
            
            #write the array back as tiff files
            
            src=rio.open(list_of_files1[0])
            with rio.Env():

                # Write an array as a raster band to a new 8-bit file. For
                # the new file's profile, we start with the profile of the source
                profile = src.profile

                # And then change the band count to 1, set the
                # dtype to uint16, and specify LZW compression.
                profile.update(
                        dtype=rio.int16,
                        count=1,
                        compress='lzw')
            originalName = list_of_files1[1].rsplit('/', 1)[-1]
            outName = outName = f"{originalName.split('.tif')[0]}"[0:19] +str('08')+ originalName[-8:]
            with rio.open(os.path.join(output_dir+outName), 'w', **profile) as dst:
                dst.write(cf.astype(rio.int16), 1)
        
 

2023-06-21 17:11:13,856 - rasterio.env - DEBUG - Entering env context: <rasterio.env.Env object at 0x7f85782d7ac0>
2023-06-21 17:11:13,859 - rasterio.env - DEBUG - Starting outermost env
2023-06-21 17:11:13,861 - rasterio.env - DEBUG - No GDAL environment exists
2023-06-21 17:11:13,864 - rasterio.env - DEBUG - New GDAL environment <rasterio._env.GDALEnv object at 0x7f8578468d90> created
2023-06-21 17:11:13,877 - rasterio._filepath - DEBUG - Installing FilePath filesystem handler plugin...
2023-06-21 17:11:13,880 - rasterio._env - DEBUG - GDAL data found in package: path='/home/romeokienzler/gitco/component-library/.venv/lib64/python3.10/site-packages/rasterio/gdal_data'.
2023-06-21 17:11:13,883 - rasterio._env - DEBUG - PROJ data found in package: path='/home/romeokienzler/gitco/component-library/.venv/lib64/python3.10/site-packages/rasterio/proj_data'.
2023-06-21 17:11:13,885 - rasterio._env - DEBUG - Started GDALEnv: self=<rasterio._env.GDALEnv object at 0x7f8578468d90>.
2023-06-21 1

For mean or max calculations, same iterations but replace 
"for i in np.arange(len(stacked_list)-1):  
                cf=xr.where((cf==0),cf_band_all[i+1],cf)""
        
with "cf=cf_band_all.mean(axis=0) or cf_band_all.max(axis=0) "